Reference: https://www.pragnakalp.com/ https://colab.research.google.com/drive/14rYdqGAXJhwVzslXT4XIwNFBwkmBWdVV#scrollTo=E6HQS-mNlN9V

### 5 news group

In [1]:
import pandas as pd
import os
import numpy as np


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline


In [1]:
!git clone https://github.com/kamalkraj/BERT-NER.git


In [2]:
cd BERT-NER/

/home/sid015/DSC180/BERT-NER


In [ ]:
!pip3 install -r requirements.txt

In [47]:
!python run_ner.py --data_dir=data/ --bert_model=bert-base-cased --task_name=ner --output_dir=out_ner --max_seq_length=128 --do_train --num_train_epochs 3 --do_eval --warmup_proportion=0.1

02/13/2021 20:54:15 - INFO - __main__ -   device: cuda n_gpu: 1, distributed training: False, 16-bits training: False
02/13/2021 20:54:15 - INFO - pytorch_transformers.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt not found in cache or force_download set to True, downloading to /tmp/tmpj2x71kof
100%|████████████████████████████████| 213450/213450 [00:00<00:00, 796128.35B/s]
02/13/2021 20:54:16 - INFO - pytorch_transformers.file_utils -   copying /tmp/tmpj2x71kof to cache at /tmp/xdg-cache/torch/pytorch_transformers/5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1
02/13/2021 20:54:16 - INFO - pytorch_transformers.file_utils -   creating metadata file for /tmp/xdg-cache/torch/pytorch_transformers/5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1
02/13/2021 20:54:16 - INFO - pytorch_t

In [4]:
pwd

'/home/sid015/DSC180'

In [49]:
%%writefile bert.py
"""BERT NER Inference."""

from __future__ import absolute_import, division, print_function

import json
import os

import torch
import torch.nn.functional as F
from nltk import word_tokenize
from pytorch_transformers import (BertConfig, BertForTokenClassification,
                                  BertTokenizer)


class BertNer(BertForTokenClassification):

    def forward(self, input_ids, token_type_ids=None, attention_mask=None, valid_ids=None):
        sequence_output = self.bert(input_ids, token_type_ids, attention_mask, head_mask=None)[0]
        batch_size,max_len,feat_dim = sequence_output.shape
        valid_output = torch.zeros(batch_size,max_len,feat_dim,dtype=torch.float32,device='cuda' if torch.cuda.is_available() else 'cpu')
        for i in range(batch_size):
            jj = -1
            for j in range(max_len):
                    if valid_ids[i][j].item() == 1:
                        jj += 1
                        valid_output[i][jj] = sequence_output[i][j]
        sequence_output = self.dropout(valid_output)
        logits = self.classifier(sequence_output)
        return logits

class Ner:

    def __init__(self,model_dir: str):
        self.model , self.tokenizer, self.model_config = self.load_model(model_dir)
        self.label_map = self.model_config["label_map"]
        self.max_seq_length = self.model_config["max_seq_length"]
        self.label_map = {int(k):v for k,v in self.label_map.items()}
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model = self.model.to(self.device)
        self.model.eval()

    def load_model(self, model_dir: str, model_config: str = "model_config.json"):
        model_config = os.path.join(model_dir,model_config)
        model_config = json.load(open(model_config))
        model = BertNer.from_pretrained(model_dir)
        tokenizer = BertTokenizer.from_pretrained(model_dir, do_lower_case=model_config["do_lower"])
        return model, tokenizer, model_config

    def tokenize(self, text: str):
        """ tokenize input"""
        words = word_tokenize(text)
        tokens = []
        valid_positions = []
        for i,word in enumerate(words):
            token = self.tokenizer.tokenize(word)
            tokens.extend(token)
            for i in range(len(token)):
                if i == 0:
                    valid_positions.append(1)
                else:
                    valid_positions.append(0)
        return tokens, valid_positions

    def preprocess(self, text: str):
        """ preprocess """
        tokens, valid_positions = self.tokenize(text)
        ## insert "[CLS]"
        tokens.insert(0,"[CLS]")
        valid_positions.insert(0,1)
        ## insert "[SEP]"
        tokens.append("[SEP]")
        valid_positions.append(1)
        segment_ids = []
        for i in range(len(tokens)):
            segment_ids.append(0)
        input_ids = self.tokenizer.convert_tokens_to_ids(tokens)
        input_mask = [1] * len(input_ids)
        while len(input_ids) < self.max_seq_length:
            input_ids.append(0)
            input_mask.append(0)
            segment_ids.append(0)
            valid_positions.append(0)
        return input_ids,input_mask,segment_ids,valid_positions

    def predict(self, text: str):
        input_ids,input_mask,segment_ids,valid_ids = self.preprocess(text)
        input_ids = torch.tensor([input_ids],dtype=torch.long,device=self.device)
        input_mask = torch.tensor([input_mask],dtype=torch.long,device=self.device)
        segment_ids = torch.tensor([segment_ids],dtype=torch.long,device=self.device)
        valid_ids = torch.tensor([valid_ids],dtype=torch.long,device=self.device)
        with torch.no_grad():
            logits = self.model(input_ids, segment_ids, input_mask,valid_ids)
        logits = F.softmax(logits,dim=2)
        logits_label = torch.argmax(logits,dim=2)
        logits_label = logits_label.detach().cpu().numpy().tolist()[0]

        logits_confidence = [values[label].item() for values,label in zip(logits[0],logits_label)]

        logits = []
        pos = 0
        for index,mask in enumerate(valid_ids[0]):
            if index == 0:
                continue
            if mask == 1:
                logits.append((logits_label[index-pos],logits_confidence[index-pos]))
            else:
                pos += 1
        logits.pop()

        labels = [(self.label_map[label],confidence) for label,confidence in logits]
        words = word_tokenize(text)
        assert len(labels) == len(words)

        Person = []
        Location = []
        Organization = []
        Miscelleneous = []

        for word, (label, confidence) in zip(words, labels):
            if label=="B-PER" or label=="I-PER":
                Person.append(word)
            elif label=="B-LOC" or label=="I-LOC":
                Location.append(word)
            elif label=="B-ORG" or label=="I-ORG":
                Organization.append(word)
            elif label=="B-MISC" or label=="I-MISC":
                Miscelleneous.append(word)
            else:
                output = None

        output = []
        for word, (label, confidence) in zip(words, labels):      
            if label == "B-PER":
                output.append(' '.join(Person) + ": Person")
            if label=="B-LOC":
                output.append(' '.join(Location) + ": Location")
            if label=="B-MISC":
                output.append(' '.join(Miscelleneous) + ": Miscelleneous Entity")
            if label=="B-ORG":
                output.append(' '.join(Organization) + ": Organization")
                
        return output

Overwriting bert.py


In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/sid015/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [262]:

from bert import Ner
model = Ner("out_ner/")



In [15]:
tf=pd.read_csv('summary_sentences.csv')

tf.summary_sentences

0        TimeWarner said fourth quarter sales rose 2% t...
1        For the full-year, TimeWarner posted a profit ...
2        Quarterly profits at US media giant TimeWarner...
3        However, the company said AOL's underlying pro...
4        Its profits were buoyed by one-off gains which...
                               ...                        
17716    I am actually convinced at this point that the...
17717    Scare-mongering articles about "addictive vide...
17718    When I was made redundant I told my partner I ...
17719    And more and more so I find I have less and le...
17720    They are all engineered in such a way that ear...
Name: summary_sentences, Length: 17721, dtype: object

In [263]:
ne=[]
for sentence in tf.summary_sentences.values:
    ne.append(model.predict(sentence))

### 20 News Group Data Set

In [2]:



# datasets = load_dataset("conll2003")
# import pandas as pd
# import numpy as np
# df=pd.concat([pd.DataFrame(datasets['train']),(pd.DataFrame(datasets['validation'])),(pd.DataFrame(datasets['test']))])

In [20]:
import re
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"
digits = "([0-9])" 
def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    text = re.sub(digits + "[.]" + digits,"\\1<prd>\\2",text)
    if "i.e." in text: text = text.replace("i.e.","i<prd>e<prd>")
    if "e.g." in text:text = text.replace("e.g.","e<prd>g<prd>")
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

In [21]:
from sklearn.datasets import fetch_20newsgroups
import json

def regex_condi(string):
    if ".com" in string:
        return False
    if ".edu" in string:
        return False
    if "@" in string:
        return False
    if 'Host'in string:
        return False
    if ".gov" in string:
        return False
    return True

def clean_text(string):
    string = " ".join([i for i in string.split() if regex_condi(i)])
    string = re.sub(r"Re:","", string)
    string = re.sub(r"Reply-To:","", string)
    toreturn = string.strip()
    return toreturn

In [22]:
newsgroups_train = fetch_20newsgroups(subset='train')
news_df = pd.DataFrame.from_dict(newsgroups_train,'index').T
news_df

,data,filenames,target_names,target,DESCR
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,/home/sid015/scikit_learn_data/20news_home/20n...,alt.atheism,7,.
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,/home/sid015/scikit_learn_data/20news_home/20n...,comp.graphics,4,.
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,/home/sid015/scikit_learn_data/20news_home/20n...,comp.os.ms-windows.misc,4,
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,/home/sid015/scikit_learn_data/20news_home/20n...,comp.sys.ibm.pc.hardware,1,_
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,/home/sid015/scikit_learn_data/20news_home/20n...,comp.sys.mac.hardware,14,2
...,...,...,...,...,...
11309,From: jim.zisfein@factory.com (Jim Zisfein) \n...,/home/sid015/scikit_learn_data/20news_home/20n...,None,13,None
11310,From: ebodin@pearl.tufts.edu\nSubject: Screen ...,/home/sid015/scikit_learn_data/20news_home/20n...,None,4,None
11311,From: westes@netcom.com (Will Estes)\nSubject:...,/home/sid015/scikit_learn_data/20news_home/20n...,None,3,None
11312,From: steve@hcrlgw (Steven Collins)\nSubject: ...,/home/sid015/scikit_learn_data/20news_home/20n...,None,1,None


In [23]:
news_df.data = news_df.data.apply(lambda x: clean_text(x)+ '.')
news_df=news_df.reset_index()

In [24]:
news_df['summary_sentences']=news_df.data.apply(lambda x:split_into_sentences(x))
# df['text_sentences']=df.summary.apply(lambda x:split_into_sentences(x))
df_summary_sentences=news_df.summary_sentences.explode().reset_index()
df_summary_sentences=df_summary_sentences.merge(news_df[['index','target']], on="index", how = 'left')


In [9]:
df_summary_sentences.summary_sentences

0         From: (where's my thing) Subject: WHAT car is ...
1                                                         ?
2         Organization: University of Maryland, College ...
3         It was a 2-door sports car, looked to be from ...
4                                 It was called a Bricklin.
                                ...                        
261146                                  Guess I'll miss it.
261147                          :-((( Help me find my baby!
261148                                                    !
261149                                                    !
261150                                                 kjg.
Name: summary_sentences, Length: 261151, dtype: object

In [10]:
from tqdm import tqdm
ne2=[]
for sentence in tqdm(df_summary_sentences.summary_sentences.values):
    try:
        if len(sentence)<512:
            ne2.append(model.predict(sentence))
        else:
            ne2.append([])
    except (AssertionError,TypeError):
        ne2.append([])

100%|██████████| 261151/261151 [2:03:34<00:00, 35.22it/s]  


In [11]:
df_summary_sentences['NE']=ne2

In [12]:
df_summary_sentences.to_csv('raw.csv',index=False)
